### Estimating MLE through PyTorch

- Generate data from a Poisson regression
- Create a loglikelihood function for the parameters
- Create the score function for the parameters using autograd on the loglikelihood
- Create the Hessian function for the parameters using autograd on the score function
- Use gradient descent to find MLE estimates and invert Hessian to estimate SEs
- Compare results with estimates from GLM package




In [82]:
# Packages
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data
n = 10000
x = np.random.uniform(0,1,(n,2))
beta = np.array([1.0,1.8])
lmbda = np.exp(np.dot(x, beta))
y = np.random.poisson(lmbda, n)
print(beta, lmbda[0:5])
print(np.c_[y, x][0:5])

[1.  1.8] [10.38737376  5.61436557  4.25111613  3.76185124  3.52810146]
[[ 8.          0.66646649  0.93006917]
 [10.          0.45443313  0.70605304]
 [ 1.          0.30554685  0.63424151]
 [ 1.          0.58640304  0.4102823 ]
 [ 4.          0.0520169   0.67152389]]


In [83]:
beta = torch.tensor(beta)
x = torch.tensor(x)
y = torch.tensor(y)

def neglogLik(beta, x, y):
    """Constructs the loglikelihood of the parameters given the data"""
    log_lambda = torch.matmul(x, beta)
    negloglikelihood = -torch.sum(y * log_lambda - torch.exp(log_lambda))
    # R broadcasting a * b differ from Python broadcasting rules
    return negloglikelihood

def score(beta):
    """Computes the gradient of the loglikelihood i.e. the score vector"""
    beta_tensor = torch.tensor(beta, requires_grad = True)
    negloglikelihood = neglogLik(beta_tensor, x, y)
    score = torch.autograd.grad(negloglikelihood, beta_tensor)[0]
    return score

def hessian(beta):
    """Computes the second derivative of the loglikelihood function i.e. the Hessian matrix"""
    beta_tensor = torch.tensor(beta, requires_grad = True)
    negloglikelihood = lambda b: neglogLik(b, x, y)
    hess = torch.autograd.functional.hessian(negloglikelihood, beta_tensor)
    return hess

def varcov(beta):
    """Inverts the Hessian to get the Standard Errors"""
    fisherInformation = hessian(beta)
    stderr = torch.sqrt(torch.diagonal(torch.linalg.inv(fisherInformation)))
    return stderr.numpy()

def evalLoss(beta):
    negloglikelihood = neglogLik(torch.tensor(beta), x, y).item()
    return negloglikelihood

def evalGrad(beta):
    grad = score(torch.tensor(beta))
    return grad

def evalHess(beta):
    hess = hessian(torch.tensor(beta))
    return hess

print('neglogLik', neglogLik(beta, x, y))
print('score', score(beta))
print('hessian', hessian(beta))
print('loss',evalLoss(beta))
print('grad',evalGrad(beta))
print('hess',evalHess(beta))
print('varcov',varcov(beta+0.01))

neglogLik tensor(-36038.6777, dtype=torch.float64)
score tensor([-143.7124, -188.2054], dtype=torch.float64)
hessian tensor([[20033.6765, 17983.0118],
        [17983.0118, 23308.0256]], dtype=torch.float64)
loss -36038.67768981711
grad tensor([-143.7124, -188.2054], dtype=torch.float64)
hess tensor([[20033.6765, 17983.0118],
        [17983.0118, 23308.0256]], dtype=torch.float64)
varcov [0.01266956 0.01174804]


In [84]:
print(beta)

from scipy.optimize import minimize
beta_init = np.array([1,2])
res = minimize(evalLoss, beta_init, tol = 1e-12, method='BFGS', jac=evalGrad)
print(res.x)
print(varcov(res.x))

from scipy.optimize import minimize
beta_init = np.array([0,0])
res = minimize(evalLoss, beta_init, tol = 1e-12, method='Newton-CG', jac=evalGrad, hess=evalHess)
print(res.x)
print(varcov(res.x))

tensor([1.0000, 1.8000], dtype=torch.float64)
[0.99976332 1.80823107]
[0.01271626 0.01178099]
[0.99976332 1.80823107]
[0.01271626 0.01178099]


### Evaluate

In [85]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
df = pd.DataFrame(np.c_[y,x], columns = ['y','x1', 'x2'])
print(smf.glm('y ~ x1 + x2 - 1', df, family=sm.families.Poisson()).fit().summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -21000.
Date:                Thu, 11 Jan 2024   Deviance:                       10886.
Time:                        11:18:24   Pearson chi2:                 1.02e+04
No. Iterations:                     5   Pseudo R-squ. (CS):             0.7916
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9998      0.013     78.621      0.0